In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

cover_column_names=["Elevation","Aspect","Slope",
              "Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology",
              "Horizontal_Distance_To_Roadways","Hillshade_9am","Hillshade_Noon",
              "Hillshade_3pm","Horizontal_Distance_To_Fire_Points"]
wilderness_cols =["Wilderness_"+str(i) for i in range(4)]
soil_col =["Soil_"+str(i) for i in range(40)]
cover_columns_names = cover_column_names + wilderness_cols + soil_col

COVER_FILE = './forest_cover_dataset.csv'
forest_dataFile = pd.read_csv(COVER_FILE,names=cover_columns_names)
#Determine the number and types of features.
forest_dataFile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280251 entries, 0 to 280250
Data columns (total 54 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           280251 non-null  int64
 1   Aspect                              280251 non-null  int64
 2   Slope                               280251 non-null  int64
 3   Horizontal_Distance_To_Hydrology    280251 non-null  int64
 4   Vertical_Distance_To_Hydrology      280251 non-null  int64
 5   Horizontal_Distance_To_Roadways     280251 non-null  int64
 6   Hillshade_9am                       280251 non-null  int64
 7   Hillshade_Noon                      280251 non-null  int64
 8   Hillshade_3pm                       280251 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  280251 non-null  int64
 10  Wilderness_0                        280251 non-null  int64
 11  Wilderness_1                        280251 non-null 

In [2]:
forest_dataFile.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_30,Soil_31,Soil_32,Soil_33,Soil_34,Soil_35,Soil_36,Soil_37,Soil_38,Soil_39
0,2825,265,17,319,108,2298,174,245,209,342,...,0,0,1,0,0,0,0,0,0,0
1,2719,198,34,134,76,2352,187,245,167,693,...,0,0,0,0,0,0,0,0,0,0
2,3146,152,14,212,41,940,237,239,130,2007,...,0,0,0,0,0,0,0,0,0,0
3,2882,18,18,95,-3,485,203,199,128,342,...,1,0,0,0,0,0,0,0,0,0
4,2912,349,17,283,57,2729,187,210,160,2201,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#Perform a ShuffleSplit of the data into training/validation/test sets, 60%/20%/20%.
from sklearn.model_selection import ShuffleSplit

def MySplit(n_splits,test_size,df):
  split = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=23721266)
  for train_index, test_valid_index in split.split(df):
     set1 = df.iloc[train_index]
     set2 = df.iloc[test_valid_index]
     return set1,set2

train_set,test_valid_set = MySplit(1,0.4,forest_dataFile)
test_set,valid_set = MySplit(1,0.5,test_valid_set)

In [6]:
#Center the training/validation/test splits (fit on the training set, then transform the validation/test sets).
from sklearn.preprocessing import StandardScaler
center_function = lambda x: x - x.mean()
scaler = StandardScaler(with_std=False)

'''
train_set_centered = train_set.apply(center_function)
test_set_centered = test_set.apply(center_function)
valid_set_centered = valid_set.apply(center_function)
'''


scaler.fit(train_set)
train_set_centered = pd.DataFrame(scaler.transform(train_set),columns = train_set.columns, index=train_set.index)
test_set_centered = pd.DataFrame(scaler.transform(test_set),columns = test_set.columns, index=test_set.index)
valid_set_centered = pd.DataFrame(scaler.transform(valid_set),columns = valid_set.columns, index=valid_set.index)


train_set.head()
train_set_centered.head()

'\ntrain_set_centered = train_set.apply(center_function)\ntest_set_centered = test_set.apply(center_function)\nvalid_set_centered = valid_set.apply(center_function)\n'

StandardScaler(with_std=False)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_30,Soil_31,Soil_32,Soil_33,Soil_34,Soil_35,Soil_36,Soil_37,Soil_38,Soil_39
273258,3013,177,10,361,56,4992,225,246,153,1559,...,0,0,0,0,0,0,0,0,0,0
165570,3163,41,16,30,10,1536,221,204,115,2462,...,0,0,0,0,0,0,0,0,0,0
146527,3214,144,14,270,27,2126,239,237,124,1708,...,0,0,0,0,0,0,0,0,0,0
79044,2321,352,26,42,12,1208,165,185,152,277,...,0,0,0,0,0,0,0,0,0,0
128539,2968,167,12,256,49,5419,230,244,145,2995,...,0,0,0,0,0,0,0,0,0,0


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_30,Soil_31,Soil_32,Soil_33,Soil_34,Soil_35,Soil_36,Soil_37,Soil_38,Soil_39
273258,68.25479,21.011882,-4.136319,94.218281,10.302611,2655.780303,13.071145,22.630318,10.19281,-419.167939,...,-0.044966,-0.092382,-0.078828,-0.002652,-0.00176,-0.000077,0.0,-0.016509,-0.014535,-0.009051
165570,218.25479,-114.988118,1.863681,-236.781719,-35.697389,-800.219697,9.071145,-19.369682,-27.80719,483.832061,...,-0.044966,-0.092382,-0.078828,-0.002652,-0.00176,-0.000077,0.0,-0.016509,-0.014535,-0.009051
146527,269.25479,-11.988118,-0.136319,3.218281,-18.697389,-210.219697,27.071145,13.630318,-18.80719,-270.167939,...,-0.044966,-0.092382,-0.078828,-0.002652,-0.00176,-0.000077,0.0,-0.016509,-0.014535,-0.009051
79044,-623.74521,196.011882,11.863681,-224.781719,-33.697389,-1128.219697,-46.928855,-38.369682,9.19281,-1701.167939,...,-0.044966,-0.092382,-0.078828,-0.002652,-0.00176,-0.000077,0.0,-0.016509,-0.014535,-0.009051
128539,23.25479,11.011882,-2.136319,-10.781719,3.302611,3082.780303,18.071145,20.630318,2.19281,1016.832061,...,-0.044966,-0.092382,-0.078828,-0.002652,-0.00176,-0.000077,0.0,-0.016509,-0.014535,-0.009051


In [5]:
#𝑘 -Means Clustering
from sklearn.cluster import KMeans

from joblib import Parallel, delayed, parallel_backend

with parallel_backend("loky", inner_max_num_threads=2):
    results = Parallel(n_jobs=4)(delayed(func)(x, y) for x, y in data)
    
k_values=list(range(2,11))+[15,20]

kmeans = KMeans(n_clusters=2, random_state=0).fit(train_set)

NameError: name 'data' is not defined